In [ ]:
#ran this notebook on Ostrich
#first subset allc file containing some CG dinucleotides where both strands are represented

In [1]:
pwd

'/Users/strigg/Documents/GitHub/Shelly_Pgenerosa/analyses/JuviPgen_ALSL2lowd145/Get_DMRs_for_SR_v074_alignments/methylpy_DMRs/destrand_allc_files'

In [3]:
%%bash
zcat < ../allc_EPI-205_S26_L004_R1_001_val_1_bismark_bt2_pe.tsv.gz | \
grep PGA_scaffold1__ | \
awk '{if(($2<2401000)&&($2>2400000))print $0}' | \
gzip > allc_test.tsv.gz

In [ ]:
#above opens allc file
#then grep selects lines from scaffold 1 only
#then awk command selects lines with chromosome positions between 2401000 and 2400000
#I chose that window because I had seen some CG dinucleotides that had both strands represented

In [ ]:
#the fields of allc files are as follows:
#1. chromosome
#2. start position
#3. strand
#4. cytosine context
#5. number of methylated cytosines
#6. number of unmethylated cytosines
#7. statistical significance of methylated C (based on lambda BS conversion control; default output is 1 when no control is detected in sample )
#for more detail, refer to https://github.com/yupenghe/methylpy/blob/methylpy/tutorial/tutorial.md

In [4]:
!zcat < allc_test.tsv.gz | head

PGA_scaffold1__77_contigs__length_89643857	2400002	-	CTC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400005	-	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400010	-	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400011	-	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400012	-	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400016	+	CAT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400024	+	CAA	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400028	+	CTT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400032	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400034	-	CAG	0	2	1


In [ ]:
#For example CG at 2400983-2400984 has both strands represented

In [5]:
!zcat < allc_test.tsv.gz | tail -11

PGA_scaffold1__77_contigs__length_89643857	2400983	+	CGC	2	2	1
PGA_scaffold1__77_contigs__length_89643857	2400984	-	CGT	1	1	1
PGA_scaffold1__77_contigs__length_89643857	2400985	+	CGT	2	2	1
PGA_scaffold1__77_contigs__length_89643857	2400986	-	CGC	1	1	1
PGA_scaffold1__77_contigs__length_89643857	2400990	+	CCG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400991	+	CGA	2	2	1
PGA_scaffold1__77_contigs__length_89643857	2400992	-	CGG	1	1	1
PGA_scaffold1__77_contigs__length_89643857	2400994	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400996	-	CAG	0	1	1
PGA_scaffold1__77_contigs__length_89643857	2400998	-	CTC	0	1	1
PGA_scaffold1__77_contigs__length_89643857	2400999	+	CTT	0	2	1


In [ ]:
#The methylkit destrand function (line 651-678 
#of this code: https://github.com/al2na/methylKit/blob/master/R/methylDBFunctions.R )
#uses the code ".CpG.dinuc.unify" which I found documentation for here:
# (https://rdrr.io/bioc/methylKit/src/R/backbone.R)
# the function subtracts 1 from start and end positions of cpgs on the reverse strand
# then merges the forward and reverse CpG based on chromosome, start, and end positions
# and then sums the coverage of forward and reverse CpG
# see code excerpt below:

    # .CpG.dinuc.unifyOld<-function(cpg)
    # {

    #   cpgR=cpg[cpg$strand=="-",]
    #   cpgF=cpg[cpg$strand=="+",]
    #   cpgR$start=cpgR$start-1
    #   cpgR$end=cpgR$end-1
    #   cpgR$strand="+"

    #   #cpgR$id=paste(cpgR$chr,cpgR$start,sep=".")

    #   cpgFR=merge(cpgF,cpgR,by=c("chr","start","end"))
    #   #hemi =cpgFR[abs(cpgFR$freqC.x-cpgFR$freqC.y)>=50,]
    #   #cpgFR=cpgFR[abs(cpgFR$freqC.x-cpgFR$freqC.y)<50,]  
    #   res=data.frame(
    #     chr     =as.character(cpgFR$chr),
    #     start    =as.integer(cpgFR$start),
    #     end      =as.integer(cpgFR$start),
    #     strand  =rep("+",nrow(cpgFR)),
    #     coverage=cpgFR$coverage.x + cpgFR$coverage.y,
    #     numCs   =cpgFR$numCs.x + cpgFR$numCs.y ,
    #     numTs   =cpgFR$numTs.x + cpgFR$numTs.y ,stringsAsFactors =FALSE
    #   )
    #   Fid=paste(cpgF$chr,cpgF$start,cpgF$end)
    #   Rid=paste(cpgR$chr,cpgR$start,cpgR$end)
    #   resid=paste(res$chr,res$start,res$end)  
    #   res=rbind(res, cpgF[ !  Fid  %in%  resid,],cpgR[ ! Rid  %in%  resid,] )
    #   res=res[order(res$chr,res$start),]
    #   return(res)
    # }


In [8]:
#based on the methylkit code, subtract 1 from all reverse strand CpG start positions
#awk code below says if field 3 (strand) is "-" then subtract 1 from start position
#and change strand to "+" and print all other fields; if field 3 is "+" then print
#line without any changes


In [9]:
!zcat < allc_test.tsv.gz | \
awk -F"\t" '{if($3=="-")print $1FS$2-1FS"+"FS$4FS$5FS$6FS$7;else print $0}' 

PGA_scaffold1__77_contigs__length_89643857	2400001	+	CTC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400004	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400009	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400010	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400011	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400016	+	CAT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400024	+	CAA	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400028	+	CTT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400032	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400033	+	CAG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400037	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400038	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400039	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400040	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400041	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400042	+	CT

In [ ]:
#looks good

#we can print the first 3 fields, sort, uniq -c, and reverse sort
#to see which cytosines had representation on both strands
#Only the lines with 2 counts in the first column are positions 
#that had representation from both strands

In [12]:
!zcat < allc_test.tsv.gz | \
awk -F"\t" '{if($3=="-")print $1FS$2-1FS"+"FS$4FS$5FS$6FS$7;else print $0}' | \
awk -F"\t" '{print $1,$2,$3}' | sort -n -k1,1 -k2,2 -k3,3 | uniq -c | sort -rn

   2 PGA_scaffold1__77_contigs__length_89643857 2400991 +
   2 PGA_scaffold1__77_contigs__length_89643857 2400985 +
   2 PGA_scaffold1__77_contigs__length_89643857 2400983 +
   2 PGA_scaffold1__77_contigs__length_89643857 2400604 +
   2 PGA_scaffold1__77_contigs__length_89643857 2400044 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400999 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400997 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400995 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400994 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400990 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400980 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400979 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400976 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400972 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400968 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400966 +
   1 PGA_scaffold1__77_contigs__length_89643857 2400965 +
   1 PGA_scaff

In [ ]:

#next I want to make the trinucleotide context the same 
# if the chromosme and start position are the same

#found some code here https://stackoverflow.com/questions/14350120/how-can-i-compare-the-columns-from-different-lines-in-awk
#that I was able to alter to the awk code below.
# to explain the code goes through line by line and checks if 
#the chromosome ($1) and start position ($2) are the same as, but the context ($4) different
#than the previous line
#if they are, 
    # if(($1==chr && $2==p && $4!=c))
#it will print the chromosome, start position, and context from the previous line
#while leaving other fields the same
    # print chr"\t"p"\t"$3"\t"c"\t"$5"\t"$6"\t"$7 
#if they aren't, it will print the line with no changes 
    # ;else print $0
#the last set of brackets is where the code stores 
#the previous line's values for chromosome (chr), start (p), and context (c) 
    # {chr=$1;p=$2; c=$4}

In [11]:
!zcat < allc_test.tsv.gz | \
awk -F"\t" '{if($3=="-")print $1FS$2-1FS"+"FS$4FS$5FS$6FS$7;else print $0}' | \
awk -F"\t" '{if(($1==chr && $2==p && $4!=c))print chr"\t"p"\t"$3"\t"c"\t"$5"\t"$6"\t"$7;else print $0}{chr=$1;p=$2; c=$4}'

PGA_scaffold1__77_contigs__length_89643857	2400001	+	CTC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400004	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400009	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400010	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400011	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400016	+	CAT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400024	+	CAA	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400028	+	CTT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400032	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400033	+	CAG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400037	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400038	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400039	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400040	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400041	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400042	+	CT

In [ ]:
# we can see that for all positions with representation from both strands
# (2400044, 2400604, 2400983, 2400985, and 2400991)
# both lines now have the same cytosine context, while other lines with unique 
# cytosine positions have their original context

In [ ]:
#next we need to collapse lines with common 
#chromosomse, start position, strand, and context AND sum their methylated C's
# and sum their unmethylated C's.
#and finally sort the output
#explanation of awk code:
# create an array of fields 1-4 that is equal to the sum of methylated C's ($5) 
     # a[$1"\t"$2"\t"$3"\t"$4]+=$5
# create another array of fields 1-4 that is equal to the sum of unmethylated C's ($6) 
     # a[$1"\t"$2"\t"$3"\t"$4]+=$6
#for each entry in the array print
# the entry (fields 1-4), 
# a[i] which is the sum of methylated C's for the entry
# b[i] which is the sum of unmethylated C's for the entry
# and 1 for the seventh field of the allc

#the sort function sorts numerically (-n) on fields 1 (-k1,1) and 2 (-k2,2)

In [18]:
!zcat < allc_test.tsv.gz | \
awk -F"\t" '{if($3=="-")print $1FS$2-1FS"+"FS$4FS$5FS$6FS$7;else print $0}' | \
awk -F"\t" '{if(($1==chr && $2==p && $4!=c))print chr"\t"p"\t"$3"\t"c"\t"$5"\t"$6"\t"$7;else print $0}{chr=$1;p=$2; c=$4}'| \
awk -F"\t" '{a[$1"\t"$2"\t"$3"\t"$4]+=$5; b[$1"\t"$2"\t"$3"\t"$4]+=$6}END{for(i in a)print i"\t"a[i]"\t"b[i]"\t"1}' | \
sort -n -k1,1 -k2,2 \
> allc_test.destrand.tsv


In [19]:
!cat < allc_test.destrand.tsv

PGA_scaffold1__77_contigs__length_89643857	2400001	+	CTC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400004	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400009	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400010	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400011	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400016	+	CAT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400024	+	CAA	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400028	+	CTT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400032	+	CTG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400033	+	CAG	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400037	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400038	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400039	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400040	+	CCC	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400041	+	CCT	0	2	1
PGA_scaffold1__77_contigs__length_89643857	2400042	+	CT

In [ ]:
#We can check to see if positions
# 2400044, 2400604, 2400983, 2400985, and 2400991
# have correctly been collapsed

#first do a line count


In [14]:
!zcat < allc_test.tsv.gz | \
wc -l


     177


In [20]:
!wc -l allc_test.destrand.tsv


     172 allc_test.destrand.tsv


In [ ]:
# 5 lines have been removed so that's a good sign

In [ ]:
# next let's look at the original data and the new data side by size,
# but remove chromosome info since we know the chromosome is the same for all lines

In [28]:
!zcat < allc_test.tsv.gz | paste -d "\t" - allc_test.destrand.tsv | awk '{$1=$8="";print $0}'


 2400002 - CTC 0 2 1  2400001 + CTC 0 2 1
 2400005 - CTG 0 2 1  2400004 + CTG 0 2 1
 2400010 - CTG 0 2 1  2400009 + CTG 0 2 1
 2400011 - CCT 0 2 1  2400010 + CCT 0 2 1
 2400012 - CCC 0 2 1  2400011 + CCC 0 2 1
 2400016 + CAT 0 2 1  2400016 + CAT 0 2 1
 2400024 + CAA 0 2 1  2400024 + CAA 0 2 1
 2400028 + CTT 0 2 1  2400028 + CTT 0 2 1
 2400032 + CTG 0 2 1  2400032 + CTG 0 2 1
 2400034 - CAG 0 2 1  2400033 + CAG 0 2 1
 2400037 + CCC 0 2 1  2400037 + CCC 0 2 1
 2400038 + CCC 0 2 1  2400038 + CCC 0 2 1
 2400039 + CCC 0 2 1  2400039 + CCC 0 2 1
 2400040 + CCC 0 2 1  2400040 + CCC 0 2 1
 2400041 + CCT 0 2 1  2400041 + CCT 0 2 1
 2400042 + CTC 0 2 1  2400042 + CTC 0 2 1
 2400044 + CGT 1 2 1  2400044 + CGT 3 4 1
 2400045 - CGA 2 2 1  2400063 + CGG 4 4 1
 2400063 + CGG 4 4 1  2400069 + CAA 0 4 1
 2400069 + CAA 0 4 1  2400072 + CTA 0 4 1
 2400072 + CTA 0 4 1  2400075 + CAT 0 5 1
 2400075 + CAT 0 5 1  2400083 + CTT 0 5 1
 2400083 + CTT 0 5 1  2400086 + CAA 0 5 1
 2400086 + CAA 0 5 1  2400089 + CT

In [ ]:
# everything looks good! hopefully it works on all the allc files!